In [15]:
# Loading all the needed Packages
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# PROBLEM 1

## 1) Data Handling

In [2]:
## PREPARING THE DATA

# Load all CSV files
Dem_Data = pd.read_excel('../Data/LoadProfile.xlsx', sheet_name='Python_Dem_Data')
Uti_Data = pd.read_excel('../Data/LoadProfile.xlsx', sheet_name='Python_Uti_Data')
Gen_E_Data = pd.read_excel('../Data/Generators_Existing.xlsx', sheet_name='Python_Gen_E_Data')
Gen_N_Data = pd.read_excel('../Data/Generators_New.xlsx', sheet_name='Python_Gen_N_Data')
Gen_E_MaxC_Data = pd.read_excel('../Data/GenerationProfile.xlsx', sheet_name='Python_Gen_E_MaxC_Data')
Gen_N_MaxC_Data = pd.read_excel('../Data/GenerationProfile.xlsx', sheet_name='Python_Gen_N_MaxC_Data')

# Export the needed matrices
Dem = np.array(Dem_Data)    # Demand profile
Uti = np.transpose(np.array(Uti_Data))   # Utility profile
Gen_E_Cost = np.array(Gen_E_Data['Cost'])   # Existing Generators Operational Cost
Gen_N_Cost = np.array(Gen_E_Data['Cost'])  # New Generators Operational Cost
Gen_E_MaxC = np.array(Gen_E_MaxC_Data)  # Maximum Capacity of Existing Generators (Hourly profile if RES, Max capacity otherwise)
Gen_N_MaxC = np.array(Gen_N_MaxC_Data)  # Maximum Capacity of New Generators (Hourly profile if RES, Max capacity otherwise)
Gen_N_MaxInvCap = np.array(Gen_N_Data['MaxInv (MW)'])  # Maximum New Generators Capacity Investment (MW)
Gen_N_InvCost = np.array(Gen_N_Data['C_CapInv ($/MW)'])  # New Generators Investment Cost ($/MW)

# Fix the shape of matrices with only one column    
Gen_E_Cost = Gen_E_Cost.reshape((Gen_E_Cost.shape[0], 1))
Gen_N_Cost = Gen_N_Cost.reshape((Gen_N_Cost.shape[0], 1))
Gen_N_MaxInvCap = Gen_N_MaxInvCap.reshape((Gen_N_MaxInvCap.shape[0], 1))
Gen_N_InvCost = Gen_N_InvCost.reshape((Gen_N_InvCost.shape[0], 1))


In [3]:
## DATA VISUALIZATION

# Create a Dataframe to store the name of each vector/matrix we will use, their size and their content
Data = pd.DataFrame(columns=['Name', 'Size', 'Content'])
Data['Name'] = ['Dem', 'Uti', 'Gen_E_Cost', 'Gen_N_Cost', 'Gen_E_MaxC', 'Gen_N_Max_C', 'Gen_N_MaxInvCap', 'Gen_N_InvCost']
Data['Size'] = [Dem.shape, Uti.shape, Gen_E_Cost.shape, Gen_N_Cost.shape, Gen_E_MaxC.shape, Gen_N_MaxC.shape, Gen_N_MaxInvCap.shape, Gen_N_InvCost.shape]
Data['Content'] = ['Demand for each load for each hour of the investment problem',
                     'Utility for each load for one hour',
                     'Unit cost of each existing generator',
                     'Unit cost of each new generator',
                     'Maximum generation profile of each existing energy source for each hour of the investment problem (Hourly profile if RES, Max capacity otherwise)',
                     'Maximum generation profile of each new energy source for each hour of the investment problem (Hourly profile if RES, Max capacity otherwise)',
                     'Maximum capacity investment of each new generator',
                     'Unit Investment cost of each new generator']
Data

,Name,Size,Content
0,Dem,"(3600, 17)",Demand for each load for each hour of the inve...
1,Uti,"(17, 1)",Utility for each load for one hour
2,Gen_E_Cost,"(16, 1)",Unit cost of each existing generator
3,Gen_N_Cost,"(16, 1)",Unit cost of each new generator
4,Gen_E_MaxC,"(3600, 16)",Maximum generation profile of each existing en...
5,Gen_N_Max_C,"(3600, 16)",Maximum generation profile of each new energy ...
6,Gen_N_MaxInvCap,"(16, 1)",Maximum capacity investment of each new generator
7,Gen_N_InvCost,"(16, 1)",Unit Investment cost of each new generator


In [4]:
## PARAMETERS DEFINITION

# Time
H = 24          # Hours in a day
D = 5           # Typical days in a year
Y = 30          # Years of the investment timeline
N = H*D*Y       # Number of hours in the investment timeline    

# Number of loads and generators
N_dem = len(Dem[0,:])       # Number of loads
N_gen_E = len(Gen_E_Cost)   # Number of existing generators
N_gen_N = len(Gen_N_Cost)   # Number of new generators

# Hyperparameters
B = 500000000    # Budget for the investment problem

# Usefull vectors
Sum_over_dem = np.ones((N_dem,1)) # Vector of ones to sum the demands over hours
Sum_over_gen_E = np.ones((N_gen_E,1)) # Vector of ones to sum the generation over hours
Sum_over_gen_N = np.ones((N_gen_N,1)) # Vector of ones to sum the generation over hours
Sum_over_hours = np.ones((N,1)) # Vector of ones to sum over hours
Sum_over_hours_gen_N = np.ones((N, N_gen_N)) # Vector of ones to sum over hours and generators

In [14]:
## CREATING A CLASS FOR PARAMETERS

class Parameters:
    def __init__(self, H, D, Y, N, N_dem, N_gen_E, N_gen_N, B, Sum_over_dem, Sum_over_gen_E, Sum_over_gen_N, Sum_over_hours, Sum_over_hours_gen_N):
        self.H = H
        self.D = D
        self.Y = Y
        self.N = N
        self.N_dem = N_dem
        self.N_gen_E = N_gen_E
        self.N_gen_N = N_gen_N
        self.B = B
        self.Sum_over_dem = Sum_over_dem
        self.Sum_over_gen_E = Sum_over_gen_E
        self.Sum_over_gen_N = Sum_over_gen_N
        self.Sum_over_hours = Sum_over_hours
        self.Sum_over_hours_gen_N = Sum_over_hours_gen_N
    

## 2) Optimisation model

### Market Clearing

In [6]:
## CREATE THE MARKET CLEARING MODEL

m = gp.Model("DA market clearing")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-24


In [7]:
## ADD VARIABLES

d = m.addMVar((N, N_dem), lb=0) # demand per hour in every demand
p_E = m.addMVar((N, N_gen_E), lb=0) # power output per hour for every existing generator

In [8]:
## ADD CONSTRAINTS

# Max demand constraint
max_dem = m.addConstr(d <= Dem, name='Maximum demand')

# Max production constraint
max_p_E = m.addConstr(p_E <= Gen_E_MaxC, name='Maximum RES production')

# Balance constraint
balance = m.addConstr(d @ Sum_over_dem == p_E @ Sum_over_gen_E, name='Demand balance')

In [9]:
## OBJECTIVE FUNCTION

objective = gp.quicksum (d @ Uti - p_E @ Gen_E_Cost)
m.setObjective(objective, GRB.MAXIMIZE)

In [10]:
m.optimize()


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 122400 rows, 118800 columns and 237600 nonzeros
Model fingerprint: 0x8b163ae8
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-02, 6e+02]
Presolve removed 118887 rows and 94209 columns
Presolve time: 0.56s
Presolved: 3513 rows, 24591 columns, 24591 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 3.513e+03 (roughly 11 MB of memory)
 Factor Ops : 3.513e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  

### Investment Problem

In [11]:
def InvestmentProblem():

    ## Create the model
    m = gp.Model("Investment problem")
    

    ## ADD VARIABLES

    P_N = m.addMVar((N_gen_N, 1), lb=0) # Invested capacity in every new generator
    p_N = m.addMVar((N, N_gen_N), lb=0) # power output per hour for every new generator
        
    
    ## ADD CONSTRAINTS

    # Capacity investment constraint
    cap_inv = m.addConstr(P_N <= Gen_N_MaxInvCap, name='Maximum capacity investment')

    # Max production constraint
    ratio_invest = (P_N.T / Gen_N_MaxInvCap.T)  # % of the maximum investment capacity invested in each new generator, size (1, N_gen_N)
    ratio_invest_hourly = Sum_over_hours_gen_N * ratio_invest  # Create a matrix of size (N, N_gen_N) with the % of the maximum investment capacity invested in each new generator for each hour
    max_p_N = m.addConstr(p_N <= Gen_N_MaxC * ratio_invest_hourly , name='Maximum RES production')

    # Budget constraint
    budget = m.addConstr(P_N.T @ Gen_N_InvCost <= B, name='Budget constraint')
    
    
    ## OBJECTIVE FUNCTION

    objective = gp.quicksum (((p_N @ Sum_over_gen_N) * balance.Pi - p_N @ Gen_N_Cost)) + B - P_N.T @ Gen_N_InvCost
    m.setObjective(objective, GRB.MAXIMIZE)
    
    return m

In [12]:
## SOLVE THE INVESTMENT PROBLEM

Investment_model = InvestmentProblem()
Investment_model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 57617 rows, 57616 columns and 59434 nonzeros
Model fingerprint: 0x21c28c8f
Coefficient statistics:
  Matrix range     [1e-04, 2e+06]
  Objective range  [1e+01, 2e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 5e+08]
Presolve removed 57617 rows and 57616 columns
Presolve time: 0.06s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e+08   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.12 seconds (0.03 work units)
Optimal objective  5.000000000e+08


In [103]:
class Data_Input_Inv:
    def __init__(self, Dem, Uti, Gen_E_Cost, Gen_N_Cost, Gen_E_MaxC, Gen_N_MaxC, Gen_N_MaxInvCap, Gen_N_InvCost):
        self.Dem = Dem
        self.Uti = Uti
        self.Gen_E_Cost = Gen_E_Cost
        self.Gen_N_Cost = Gen_N_Cost
        self.Gen_E_MaxC = Gen_E_MaxC
        self.Gen_N_MaxC = Gen_N_MaxC
        self.Gen_N_MaxInvCap = Gen_N_MaxInvCap
        self.Gen_N_InvCost = Gen_N_InvCost

In [ ]:
class InvestmentProblem:
    def __init__(self, Data, B):
        self.Data = Data
        self.B = B
        

In [86]:
## CREATE THE INVESTMENT MODEL

m1 = gp.Model("Investment")


In [87]:
## ADD VARIABLES

P_N = m1.addMVar((N_gen_N, 1), lb=0) # Invested capacity in every new generator
p_N = m1.addMVar((N, N_gen_N), lb=0) # power output per hour for every new generator

In [88]:
## ADD CONSTRAINTS

# Capacity investment constraint
cap_inv = m1.addConstr(P_N <= Gen_N_MaxInvCap, name='Maximum capacity investment')

# Max production constraint
ratio_invest = (P_N.T / Gen_N_MaxInvCap.T)  # % of the maximum investment capacity invested in each new generator, size (1, N_gen_N)
ratio_invest_hourly = Sum_over_hours_gen_N * ratio_invest  # Create a matrix of size (N, N_gen_N) with the % of the maximum investment capacity invested in each new generator for each hour
max_p_N = m1.addConstr(p_N <= Gen_N_MaxC * ratio_invest_hourly , name='Maximum RES production')

# Budget constraint
budget = m1.addConstr(P_N.T @ Gen_N_InvCost <= B, name='Budget constraint')

In [91]:
## OBJECTIVE FUNCTION

objective = gp.quicksum (((p_N @ Sum_over_gen_N) * balance.Pi * 10000000000 - p_N @ Gen_N_Cost)) + B - P_N.T @ Gen_N_InvCost/1000000
m1.setObjective(objective, GRB.MAXIMIZE)

In [92]:
## SOLVE THE MODEL

m1.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 57617 rows, 57616 columns and 59434 nonzeros
Model fingerprint: 0xbc011749
Coefficient statistics:
  Matrix range     [1e-04, 2e+06]
  Objective range  [8e-01, 1e+12]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 5e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 57616 rows and 57609 columns
Presolve time: 0.10s
Presolved: 1 rows, 7 columns, 7 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.1687500e+16   5.760000e+04   0.000000e+00      0s
      43    4.1053850e+16   0.000000e+00   0.000000e+00      0s

Solved in 43 iterations and 0.26 seconds (0.18 work units)
Optimal objective  4.105385050e+16


In [93]:
# Display the generators the model decided to invest in
for i in range(N_gen_N):
    if P_N[i].x > 0:
        print('Invest in generator', i, 'with a capacity of', P_N[i].x, 'MW')

Invest in generator 2 with a capacity of [200.] MW
Invest in generator 3 with a capacity of [200.] MW
Invest in generator 6 with a capacity of [20.] MW
Invest in generator 15 with a capacity of [200.] MW


## Sense checks

In [94]:
# Display the optimal values of the decision variables for the 5 first hours in a df
n_test = 24
df = pd.DataFrame(columns=['Hour', 'Load', 'Existing generators', 'Price'])
df['Hour'] = np.arange(1,n_test+1)
df['Load'] = d.X[0:n_test] @ Sum_over_dem
df['Existing generators'] = p_E.X[0:n_test] @ Sum_over_gen_E
df['Price'] = balance.Pi[0:n_test] 
df


,Hour,Load,Existing generators,Price
0,1,1779.385188,1779.385188,40.0
1,2,1650.595419,1650.595419,40.0
2,3,1571.538513,1571.538513,40.0
3,4,1562.649317,1562.649317,40.0
4,5,1595.064126,1595.064126,40.0
5,6,1620.124075,1620.124075,40.0
6,7,2135.303208,2135.303208,50.0
7,8,2255.132627,2255.132627,75.0
8,9,2196.182934,2196.182934,75.0
9,10,2111.414663,2111.414663,75.0


In [95]:
# Give the maximum value of p_E[:,0], p_E[:,1], p_N[:,0] and p_N[:,1] and d[:,0] and d[:,1]
print('Max value of p_E[:,0]:', max(p_E.X[:,8]))
print('Max value of p_E[:,1]:', max(p_E.X[:,9]))
print('Average value of p_E[:,0]:', np.mean(p_E.X[:,8]))
print('Average value of p_E[:,1]:', np.mean(p_E.X[:,9]))
print('Max value of d[:,0]:', max(d.X[:,0]))
print('Max value of d[:,1]:', max(d.X[:,1]))


Max value of p_E[:,0]: 283.0980684
Max value of p_E[:,1]: 199.3726899
Average value of p_E[:,0]: 116.46559658
Average value of p_E[:,1]: 36.520036802499995
Max value of d[:,0]: 186.8100487380263
Max value of d[:,1]: 165.49092384296225


In [96]:
p_E[10,:].X

array([152.        , 152.        , 350.        , 591.        ,
        60.        , 155.        , 155.        ,  21.4710252 ,
        21.4710252 ,  55.9900941 ,  16.64004453,  18.78714705,
        26.8387815 , 100.        ,  11.19801882, 200.        ])